In [1]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np

import random
import sys
import io
import re

Using TensorFlow backend.


# Preprocess

In [8]:
# path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

path = '라푼젤_수정.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

text = re.sub(r'<.*>', '', text)
text = re.sub(r'\n', ' ', text)
text = re.sub(r' +', ' ', text)

print('corpus length:', len(text))

corpus length: 3960


# Compute Character Indices

In [9]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 401


# Vectorize Sentences

In [10]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences: 1307
Vectorization...


# Create Simple Model

In [11]:
print('Build model...')
model = Sequential()
model.add(LSTM(1024, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001))

Build model...


# Define Callback

In [12]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    print('\n----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
#     for diversity in [0.2, 0.5, 1.0, 1.2]:
#         print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, 0.5)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

# Train

In [ ]:
model.fit(x, y, batch_size=128, epochs=60, callbacks=[print_callback])

Epoch 1/60
1307/1307 [==============================] - 21s 16ms/step - loss: 5.8162

----- Generating text after Epoch: 0
----- Generating with seed: " 감명을 받았답니다. 이후 왕자님은 매일 숲으로 와서 그녀의 노랫소리를 "
 감명을 받았답니다. 이후 왕자님은 매일 숲으로 와서 그녀의 노랫소리를                                 렸                                       왕                                                                           첼                                                라                                            라                              해   첼                                                                                  아                               요       
Epoch 2/60
1307/1307 [==============================] - 20s 16ms/step - loss: 4.4197

----- Generating text after Epoch: 1
----- Generating with seed: "을 시험해보리라.”라고 왕자님은 말했어요. 다음날 어둠이 깔리기 시작할 "
을 시험해보리라.”라고 왕자님은 말했어요. 다음날 어둠이 깔리기 시작할       에     가      지해      를내  에 그 가  그    그      을          가 .  어에    가    자      지하 뒤     가   을 그 그          을 지이     그      아을 

뭐가 어쩌고 어째! 너를 세상과 완전히 단절시켰는데도 이렇게 나를 속여?  아을 내 “마 상  “  아  을 마녀 내  으 마에 “녀 그  서 가  마 가 그가 “았요요. “ 에어어요. “  “  내  내 “  그  내 “으 “에 그아 왕 없에 내녀 마녀가 라래 마  마 “어 “  마 이  라 있에 “래 때어 아  는 “리 마  게 “고 “었 내  “ 이어 가 “  “는 그 .  가 마아 그  “ 첼리 그  마  고 게  마 있이 “아 가  아 주내 마  내  사 마리 “ 가   를 마  내 “기 마편 “  내 있마 마 있어 있  “ 가 다아 마 이 하아 그 게 “내 그  없  마 가 다  하아 그 게  내  이 “아 것는 “었 마 하어 그 마어 “  있 가  아 있었  래 “추 내  어 있  마녀  리게 “  내 이  마 게으 “ 하마 가 있양 마  거 를  지 있내 마녀  
Epoch 14/60
1307/1307 [==============================] - 19s 15ms/step - loss: 3.4094

----- Generating text after Epoch: 13
----- Generating with seed: "를 사랑하던 남편이 생각했어요. “아내가 이렇게 먹고 싶어 하는데 무슨 "
를 사랑하던 남편이 생각했어요. “아내가 이렇게 먹고 싶어 하는데 무슨 로에가앞을 로는가로로상로보다다로 내는을 로 을녀을 내았을로을 가 로을다로 그로 말을 그녀을 라다을로녀았을로는로을을푼로을 을는에군요을 말을 내으을을말가이로로 로로 로는다로 을는다는다지라지로을다을았보로 가을 있는을로로 을을로로로 아을 가는 말에 로 을로로을을을을로을을로는을을로로로 말를 로지을무는 가는을지을에로내로 눈는 가는 로을 내는 그다았에을 을내다는 는을을로지로을가 라을가로을로로로 로는 로는을로다을 그는 가님을 벽신을가다 을는가 아에을을로 로는 말로 로았을로을 로로 로로 있녀 로는 가녀소을로 라을 사는 가푼했을는을을자본을을로다을로로을을로을는을로을을을을로으로는가로 로로 로을

 가둔 바로 그 날 저녁 마녀는 라푼첼의 머리카락을 창의 고리에 다시 이 지는  고 처 지 있는 한 이 지 나 자 처 말 지는 안 말이 지고 자 지 지로 자님은  자님고 말라고  내지. 나 이 말 요.  라푼첼로 앞자님  지해   벽지  무신  소리고  라렇  자자님  라리고  내 는  라시니 지 라고  앞어요. 그녀를 사라고   라고  무 고 나 요고  내고 운 자리를 손내아 지  니다지 상자님  내아다고 왕자님은 볼큼.  내요.  내라  지요. 거 말 가 요.  라이  자지고 뿌 아 고 있내 지 지았로 나 나 지 요.  라푼첼을 라라고  째리를 감 아를 나 라고  고 라고 지로 자지  라 다  라해로  내리고  내리고  내주를 지고 고 지에 마고 고 말더  라지.  든리를  라다  지지는 나 머리카락을 아래  지지다  라리를  소리를 있어요. 그녀다 사 라고 로 라고 로 있
Epoch 27/60
1307/1307 [==============================] - 20s 15ms/step - loss: 2.1008

----- Generating text after Epoch: 26
----- Generating with seed: ". 다행히 죽지는 않았지만 떨어질 때 가시나무에 떨어지는 바람에 두 눈을"
. 다행히 죽지는 않았지만 떨어질 때 가시나무에 떨어지는 바람에 두 눈을 타다 자다  다이 되다 자었어요. 그라 대다 라푼첼아 상고 말 상니지  지다 자님이 뿌이 말지지 다자 왕자님은 왕자님은 상이 내 이는 되다 아으지 말이 말이 안 지는 되다 마 상어요. “다 그다다  아이 사다다. 그다. 사다 마생지  자었어요. 상자을 아지는 무지 눈 지  자을 는 이 말이 말는 마녀가 사지 는 상이 되는 이 지 지렸 지 상지 되다다 지지 다다다 다지 말지 너다지 다자 그 지는 되다지  상지  지지 지지 주자 이 말이 눈다지 지자 이 상리 상을 그 지  사 있는 되자 한 상운 되는 라다지  상다 는 이 가 머리 락을 아래다 을지다 지다 그 이 지지 말다 